In [1]:
import jax
import jax.numpy as jnp
import optax
import numpy as np
from jaxmarl.environments.coin_game.make_train import make_train

CUDA backend failed to initialize: Unable to use CUDA because of the following issues with CUDA components:
Outdated CUDA installation found.
Version JAX was built against: 12030
Minimum supported: 12010
Installed version: 12000
The local installation version must be no lower than 12010.
--------------------------------------------------
Outdated cuBLAS installation found.
Version JAX was built against: 120304
Minimum supported: 120100
Installed version: 120002
The local installation version must be no lower than 120100.
--------------------------------------------------
Outdated cuSPARSE installation found.
Version JAX was built against: 12200
Minimum supported: 12100
Installed version: 12001
The local installation version must be no lower than 12100..(Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
# Hiperparámetros
NUM_ENVS = 4
NUM_INNER_STEPS = 50
NUM_EPOCHS = 1000
NUM_AGENTS = 2
SHOW_EVERY_N_EPOCHS = 10
SAVE_EVERY_N_EPOCHS = 50
LR = 1e-2
PAYOFF_MATRIX = [[1, 1, -2], [1, 1, -2]]
GRID_SIZE = 4
REWARD_COEF = [[1, 0], [1, 0]]

In [3]:
save_dir = '/data/samuel_lozano/coin_game/'
training_type = 'AC Epoch'

config = {
    "NUM_ENVS": NUM_ENVS,
    "NUM_INNER_STEPS": NUM_INNER_STEPS,
    "NUM_EPOCHS": NUM_EPOCHS,
    "NUM_AGENTS": NUM_AGENTS,
    "SHOW_EVERY_N_EPOCHS": SHOW_EVERY_N_EPOCHS,
    "SAVE_EVERY_N_EPOCHS": SAVE_EVERY_N_EPOCHS,
    "LR": LR,
    "PAYOFF_MATRIX": PAYOFF_MATRIX,
    "GRID_SIZE": GRID_SIZE,
    "REWARD_COEF": REWARD_COEF,
    "SAVE_DIR": save_dir,
    "TRAINING_TYPE": training_type
}

params = make_train(config)


Epoch 0:
  Pure reward of agent_0 = -9.00
  Pure reward of agent_1 = 3.00


KeyboardInterrupt: 